In [97]:
from typing import List
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [98]:
def tpr(TP, FN):
    if TP + FN == 0:
        return 1
    return TP / (TP + FN)


def fpr(FP, TN):
    if FP + TN == 0:
        return 0
    return FP / (FP + TN)


def generateROC(dataset: str, inference_provider_name: str):

    fig = go.Figure()

    for criteria in [1, 2]:
        eval_res_df = (
            pd.read_csv(
                f"./results/{dataset}/{inference_provider_name}/criteria{criteria}.csv",
                header=0,
            )
            .sort_values("threshold")
            .reset_index(drop=True)
        )
        # benign_eval_res_df = (
        #     pd.read_csv(
        #         f"./results/benign/{inference_provider_name}/criteria{criteria}.csv",
        #         header=0,
        #     )
        #     .sort_values("threshold")
        #     .reset_index(drop=True)
        # )
        # eval_res_df = dataset_eval_res_df.add(benign_eval_res_df, fill_value=0)

        if criteria == 2:
            eval_res_df["FN"] = eval_res_df["FN"] + eval_res_df["TP_LATE"]

        eval_res_df["TPR"] = eval_res_df.apply(
            lambda row: tpr(row["TP"], row["FN"]), axis=1
        )
        eval_res_df["FPR"] = eval_res_df.apply(
            lambda row: fpr(row["FP"], row["TN"]), axis=1
        )

        # eval_res_df.drop(0, inplace=True)


        # eval_res_df["F1"] = eval_res_df.apply(
        #     lambda row: 2 * row["TP"] / (2 * row["TP"] + row["FP"] + row["FN"])
        #     if (2 * row["TP"] + row["FP"] + row["FN"]) > 0
        #     else 0,
        #     axis=1,
        # )

        # print("Criteria: ", criteria)
        # print(eval_res_df)
        # print("\n")
        
        eval_res_df = eval_res_df.sort_values("FPR").reset_index(drop=True)

        fig.add_trace(go.Scatter(
            x=eval_res_df["FPR"], y=eval_res_df["TPR"], name=f"Criteria {criteria}"))

    fig.add_trace(
        go.Scatter(x=[0, 1], y=[0, 1], line_color="lightgray", name="Baseline")
    )

    fig.update_layout(
        title=f"ROC Curve - {dataset} - {inference_provider_name}",
        xaxis_title="False Positive Rate",
        yaxis_title="True Positive Rate",
        yaxis_range=[0, 1],
        xaxis_range=[0, 1],
        height=400,
        width=430,
    )
    fig.update_yaxes(
        scaleanchor="x",
        scaleratio=1,
    )

    fig.show()

In [99]:

def getAUC(x: List[float], y:List[float]) -> float:
    """
    Calculates the area under the curve (AUC) using the trapezoidal rule.
    """
    if len(x) != len(y):
        raise ValueError("The number of x and y values must be equal")

    n = len(x)
    area = 0.0

    for i in range(1, n):
        # Trapezoidal rule: area of trapezoid = (1/2) * (sum of parallel sides) * height
        area += 0.5 * (x[i] - x[i-1]) * (y[i] + y[i-1])

    return area // 0.0001 / 10000

In [100]:
def generateROCForCriteria(dataset: str, criteria: int, inference_providers = ["Fake", "BartLarge", "Falcon7B", "GPT3", "EmertonMonarch7B"]):

    fig = go.Figure()

    for inference_provider_name in inference_providers:
        eval_res_df = (
            pd.read_csv(
                f"./results/{dataset}/{inference_provider_name}/criteria{criteria}.csv",
                header=0,
            )
            .sort_values("threshold")
            .reset_index(drop=True)
        )

        if criteria == 2:
            eval_res_df["FN"] = eval_res_df["FN"] + eval_res_df["TP_LATE"]

        eval_res_df["TPR"] = eval_res_df.apply(
            lambda row: tpr(row["TP"], row["FN"]), axis=1
        )
        eval_res_df["FPR"] = eval_res_df.apply(
            lambda row: fpr(row["FP"], row["TN"]), axis=1
        )
        
        eval_res_df = eval_res_df.sort_values("FPR").reset_index(drop=True)

        fin_tpr = eval_res_df["TPR"].tolist()
        fin_tpr.insert(0, 0)
        
        fin_fpr = eval_res_df["FPR"].tolist()
        fin_fpr.insert(0, 0)

        auc = getAUC(fin_fpr, fin_tpr)

        fig.add_trace(go.Scatter(
            x=fin_fpr, y=fin_tpr, name=f"{inference_provider_name} - AUC: {auc}"))

    fig.add_trace(
        go.Scatter(x=[0, 1], y=[0, 1], line_color="lightgray", name="Baseline")
    )

    fig.update_layout(
        title=f"ROC Curve - {dataset} - Criteria: {criteria}",
        xaxis_title="False Positive Rate",
        yaxis_title="True Positive Rate",
        yaxis_range=[0, 1],
        xaxis_range=[0, 1],
        height=400,
        width=580,
    )
    fig.update_yaxes(
        scaleanchor="x",
        scaleratio=1,
    )

    fig.show()

In [101]:
dataset = "hdd"

In [102]:
# generateROC(dataset, "Falcon7B")

In [103]:
# generateROC(dataset, "EmertonMonarch7B")

In [104]:
# generateROC(dataset, "GPT3")

In [105]:
# generateROC(dataset, "Fake")

In [106]:
inference_providers = ["Fake", "GPT3", "Falcon7B", "EmertonMonarch7B", "BartLarge", "Cyrax7B"]

In [107]:
dataset = "hdd"
generateROCForCriteria(dataset, 1, inference_providers )
generateROCForCriteria(dataset, 2, inference_providers )

In [108]:
dataset = "buffer-io"
generateROCForCriteria(dataset, 1, inference_providers )
generateROCForCriteria(dataset, 2, inference_providers )

In [109]:
dataset = "oom"
generateROCForCriteria(dataset, 1, inference_providers )
generateROCForCriteria(dataset, 2, inference_providers )

In [110]:
dataset = "cpu"
generateROCForCriteria(dataset, 1, inference_providers )
generateROCForCriteria(dataset, 2, inference_providers )